In [1]:
import pandas as pd
import numpy as np
import datetime as dt

import seaborn as sns
import matplotlib.pyplot as plt
import calmap
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = r'C:/Users/frede/Documents/GitHub/BC5_Demand_Forecast/data/MindOverData_RetailChallenge.csv'
pathPP = r'C:/Users/frede/Documents/GitHub/BC5_Demand_Forecast/data/values_units.csv'

In [ ]:
# 20 M
data =  pd.read_csv(path, nrows=1999998)

In [ ]:
#PreProcessing - removing excessive letters from the variables

data['ProductFamily_ID'] = data['ProductFamily_ID'].str.split('_').str[1]
data['ProductCategory_ID'] = data['ProductCategory_ID'].str.split('_').str[1]
data['ProductBrand_ID'] = data['ProductBrand_ID'].str.split('_').str[1]
data['ProductName_ID'] = data['ProductName_ID'].str.split('_').str[1]
data['ProductPackSKU_ID'] = data['ProductPackSKU_ID'].str.split('_').str[1]
data['Point-of-Sale_ID'] = data['Point-of-Sale_ID'].str.split('_').str[1]

data['Measures'] = data['Measures'].str.split(' ').str[1]

data.head(20)

In [ ]:
data.info()

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
#data = data.sort_values(by='Date')

In [ ]:
# Creating a variable with the day of the week of each purchase

data['Day of the Week'] = data['Date'].dt.dayofweek

In [ ]:
# Creating a variable with the correspondent quarter

data['Quarter'] = data['Date'].dt.quarter

In [ ]:
# Splitting the dataframe based on Units/ Values

data_units = data[data['Measures'] == 'units']
data_values = data[data['Measures'] == 'values']

In [ ]:
data_values = data_values.assign(Units=data_units['Value'].values)
data_values = data_values.drop(columns='Measures')

In [ ]:
#data_values.to_csv('values_units.csv.gz', compression='gzip')

##### Loading the preprocessed dataset

In [ ]:
dataPP =  pd.read_csv(pathPP,  nrows=100000,dtype={'ProductFamily_ID': str, 'Point-of-Sale_ID': str})

In [ ]:
#dataPP[["ProductFamily_ID", "Point-of-Sale_ID"]] = dataPP[["ProductFamily_ID", "Point-of-Sale_ID"]].astype(str) 

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="Day of the Week", data=dataPP, color='darkblue')
plt.ylabel('Count', fontsize=12)
plt.xlabel('Day of the week', fontsize=12)
plt.title("Frequency of purchase by day of the week", fontsize=15)
plt.show()

In [ ]:
cnt = dataPP['ProductName_ID'].value_counts()[:10]
plt.figure(figsize=(12,8))
sns.barplot(cnt.values, cnt.index, alpha=0.8, color='gray')
plt.xlabel('Number of Occurrences', fontsize=12)
plt.ylabel('Product ID', fontsize=12)
plt.title("Top 10 products", fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(dataPP['Value'], color='green')
plt.xlabel('Value', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.show()

### Point-of-sale

####  Quarterly analysis
###### January, February, and March (Q1)
###### April, May, and June (Q2)
###### July, August, and September (Q3)
###### October, November, and December (Q4)

In [ ]:
fig = px.line(dataPP.sort_values(by='Date'), x="Date", y="Value")
fig.show()

In [ ]:
fig = px.bar(data_values, x="total_bill", y="day", orientation='h')
fig.show()

#### Association rules

In [ ]:
dataPP =  pd.read_csv(pathPP,  nrows=1000000,dtype={'ProductFamily_ID': str, 'Point-of-Sale_ID': str})

In [ ]:
dataPP.info()

In [ ]:
dataPP['TID'] = dataPP['ProductFamily_ID'] + dataPP['Point-of-Sale_ID'] + ' ' + dataPP['Date']

In [ ]:
dataPP.head()

In [ ]:
fam = pd.pivot_table(dataPP, index='TID', columns='ProductName_ID', 
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)

In [ ]:
#fam.to_csv('fam_sample.csv.gz', compression='gzip')

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(fam, min_support=0.04, use_colnames=True)
frequent_itemsets.head()